# Regresja - przewidywanie wynagrodzenia na podstawie technologii

## Załadowanie danych z pliku csv

In [42]:
import pandas as pd
import os

df = pd.read_csv(os.path.join('..', 'jobData.csv'), on_bad_lines='skip')
df.head()

,Position,Company,Experience,Salary,Used Technologies,Optional Technologies
0,Embedded Software Engineer,Fluke Corportaion,mid,13000 18000,"{'C': 'regular', 'C++': 'regular', 'Linux': 'r...",-
1,Azure Platform Engineer (AI Department),Procter & Gamble,mid,-,"{'Microsoft Azure': 'advanced', 'Python': 'adv...",-
2,.NET Developer,UN7,mid,20000 28000,{'.Net': 'advanced'},-
3,Spec. Projektant Robotyzacji Procesów,Credit Agricole Bank Polska S.A.,mid,-,"{'C#': 'regular', 'VB.Net': 'regular', 'Java':...",-
4,HT Functional Consultant with ERP Industry Sol...,Accenture,mid,-,"{'English': 'master', 'ERP': 'advanced', 'Prob...",-


## Przygotowanie danych

In [43]:
df = df[['Salary', 'Used Technologies']]

# Wybranie wierszy tylko takich jak ten: "13000 18000"
mask = df['Salary'].str.match('^\S+\s+\S+$')
df = df[mask]

# Stworzenie kolumn Salary Min i Salary Max. W przypadku "13000 18000" Salary Min = 13000, a Salary Max = 18000
split_salaries = df['Salary'].str.split(' ', expand=True)
df['Salary_Min'] = split_salaries[0]
df['Salary_Max'] = split_salaries[1]
df['Salary_Min'] = pd.to_numeric(df['Salary_Min'], errors='coerce')
df['Salary_Max'] = pd.to_numeric(df['Salary_Max'], errors='coerce')

df = df.dropna(subset=['Salary_Min', 'Salary_Max'])
df.head()

,Salary,Used Technologies,Salary_Min,Salary_Max
0,13000 18000,"{'C': 'regular', 'C++': 'regular', 'Linux': 'r...",13000,18000.0
2,20000 28000,{'.Net': 'advanced'},20000,28000.0
10,13000 25000,"{'SQL': 'advanced', 'Python': 'advanced', 'PyS...",13000,25000.0
13,25200 31080,"{'Azure': 'advanced', 'Kubernetes': 'advanced'...",25200,31080.0
26,15000 24000,"{'Linux': 'advanced', 'Kubernetes': 'regular',...",15000,24000.0


In [44]:
import ast

def extract_technologies(tech_str):
    """ Extract technologies from a string. Assumes dictionary strings or comma-separated strings. """
    try:
        # Attempt to parse the string as a dictionary and extract keys (technologies)
        tech_dict = ast.literal_eval(tech_str)
        if isinstance(tech_dict, dict):
            return list(tech_dict.keys())
    except Exception:
        pass

    # For non-dictionary strings, assume a comma-separated list of technologies
    if isinstance(tech_str, str):
        return [tech.strip() for tech in tech_str.split(',') if tech.strip()]

    # Return an empty list for non-parsable or non-string entries
    return []

# Apply the function to each entry in the 'Used Technologies' column
df['Parsed Used Technologies'] = df['Used Technologies'].apply(extract_technologies)

# Display the first few rows of extracted technologies for verification
df.head()


,Salary,Used Technologies,Salary_Min,Salary_Max,Parsed Used Technologies
0,13000 18000,"{'C': 'regular', 'C++': 'regular', 'Linux': 'r...",13000,18000.0,"[C, C++, Linux, SVN, Yocto, Qt]"
2,20000 28000,{'.Net': 'advanced'},20000,28000.0,[.Net]
10,13000 25000,"{'SQL': 'advanced', 'Python': 'advanced', 'PyS...",13000,25000.0,"[SQL, Python, PySpark, Apache Spark, Databrick..."
13,25200 31080,"{'Azure': 'advanced', 'Kubernetes': 'advanced'...",25200,31080.0,"[Azure, Kubernetes, Python, Terraform, Gitlab ..."
26,15000 24000,"{'Linux': 'advanced', 'Kubernetes': 'regular',...",15000,24000.0,"[Linux, Kubernetes, Ansible, GitLab, Python/Ba..."


In [45]:
# Initialize a dictionary to store the new columns
tech_encoding_dict = {}

# Create a set of all unique technologies in the dataset
unique_technologies = set()
for tech_list in df['Parsed Used Technologies']:
    unique_technologies.update(tech_list)

# Populate the dictionary with binary encoding for each technology
for tech in unique_technologies:
    tech_encoding_dict[f'Used_{tech}'] = df['Parsed Used Technologies'].apply(lambda x: 1 if tech in x else 0)

# Convert the dictionary to a DataFrame
tech_encoding_df = pd.DataFrame(tech_encoding_dict)

# Concatenate the new DataFrame with the original one
df = pd.concat([df, tech_encoding_df], axis=1)
df.head()

,Salary,Used Technologies,Salary_Min,Salary_Max,Parsed Used Technologies,Used_Angular,Used_Microservices,Used_Google Cloud Platform,Used_Babel,Used_ATLAS,...,Used_NoSql,Used_Laravel,Used_Security,Used_LESS,Used_DWH,Used_Unreal Engine,Used_pyTest,Used_Terraform/Ansible,Used_MySQL,Used_Figma
0,13000 18000,"{'C': 'regular', 'C++': 'regular', 'Linux': 'r...",13000,18000.0,"[C, C++, Linux, SVN, Yocto, Qt]",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20000 28000,{'.Net': 'advanced'},20000,28000.0,[.Net],0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,13000 25000,"{'SQL': 'advanced', 'Python': 'advanced', 'PyS...",13000,25000.0,"[SQL, Python, PySpark, Apache Spark, Databrick...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,25200 31080,"{'Azure': 'advanced', 'Kubernetes': 'advanced'...",25200,31080.0,"[Azure, Kubernetes, Python, Terraform, Gitlab ...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,15000 24000,"{'Linux': 'advanced', 'Kubernetes': 'regular',...",15000,24000.0,"[Linux, Kubernetes, Ansible, GitLab, Python/Ba...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


X:

| Used_PowerShell | Used_jQuery |
|-----------------|-------------|
| 0               | 0           |
| 0               | 0           |

y:

| Salary_Min |
|------------|
| 13000      |
| 20000      |


In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import pickle

# Assuming 'Salary_Min' is the target variable
X = df.drop(['Salary_Min',
                           'Salary_Max',
                           'Salary',
                           'Used Technologies',
                           'Parsed Used Technologies'], axis=1)
y = df['Salary_Min']

with open(os.path.join('..', 'column_names.txt'), 'w') as file:
    for column_name in X.columns:
        file.write(f"{column_name}\n")

# Fit Random Forest to get feature importances
forest = RandomForestRegressor()
forest.fit(X, y)

# Select features based on importance
selector = SelectFromModel(forest, prefit=True)
X_selected = selector.transform(X)

with open(os.path.join('..', 'selector.pkl'), 'wb') as file:
    pickle.dump(selector, file)

selector

/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


SelectFromModel(estimator=RandomForestRegressor(), prefit=True)

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the model
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}, R-squared: {r2}')


MSE: 30896017.307340413, R-squared: 0.09652112938344004


## Poszukiwanie najlepszych hiperparametrów

In [48]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': [None, 'sqrt'],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Grid search
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters
print(grid_search.best_params_)


Fitting 3 folds for each of 216 candidates, totalling 648 fits


[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.3s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, max_features=None, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=10, max_features=Non

## Przewidywanie

In [49]:
# Example user input
user_input_technologies = ['SQL', 'C#']

# Initialize a feature vector with zeros
feature_vector = pd.Series(0, index=X.columns)

# Set the corresponding features to 1 based on user input
for tech in user_input_technologies:
    if f'Used_{tech}' in feature_vector.index:
        feature_vector[f'Used_{tech}'] = 1

# Transform the feature vector using the same selector as during training
# Note: We need to convert the series to a DataFrame and transpose it for the correct shape
feature_vector_transformed = selector.transform(feature_vector.to_frame().T)

# Predict the salary using the best model from GridSearchCV
best_rf_model = grid_search.best_estimator_

import pickle
pickle.dump(best_rf_model, open(os.path.join('..', 'salary_prediction_model.sav'), 'wb'))

loaded_model = pickle.load(open(os.path.join('..', 'salary_prediction_model.sav'), 'rb'))

predicted_salary_min = loaded_model.predict(feature_vector_transformed)

print("Predicted Minimum Salary: {:.2f}".format(predicted_salary_min[0]))


Predicted Minimum Salary: 15847.43


/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
